# ModOAP - Entraînement à la détection de mise en page - Layout Parser

Ce script permet d'entraîner un modèle de détection de mise en page de documents grâce à l'outil Layout Parser. 

L'entraînement doit être réalisé à partir d'un corpus de pages de documents annotées au format COCO.

Le modèle entraîné peut être sauvegardé dans un dossier Google Drive, et utilisé pour la détection avec Layout Parser.

Une fonctionnalité permet d'essayer un modèle entraîné sur une page de document et de visualiser les résultats.

**Ce script doit être lancé dans un environnement d'exécution GPU : Exécution -> Modifier le type d'exécution -> GPU**

In [ ]:
#@title Imports et synchronisation avec Drive { display-mode: "form" }

from google.colab import drive
import os

# chargement d'un google drive
if not os.path.exists("/content/drive/MyDrive/") :
  drive.mount('/content/drive/')

import shutil
import requests
import glob
import json
import cv2

try:
  import layoutparser as lp
except ModuleNotFoundError:
  !pip -q install 'git+https://github.com/facebookresearch/detectron2.git@v0.4#egg=detectron2' 
  !pip -q install -U layoutparser
  import layoutparser as lp

!git clone --quiet https://github.com/Layout-Parser/layout-model-training.git
!wget -q -O ./layout-model-training/config_LayoutParser_PrimaDataset_original.yaml https://www.dropbox.com/s/yc92x97k50abynt/config.yaml?dl=1

import torch
torch.cuda.empty_cache()

# Entrainement

In [ ]:
#@markdown ## Entrainement d'un modèle à partir d'un dataset COCO
#@markdown Renseigner les paramètres avant de lancer la cellule

#@markdown ---

#@markdown #### Entrer le chemin vers le dossier contenant le corpus d'entraînement au format COCO :
chemin_corpus = "" #@param {type:"string"}
#@markdown Exemple de chemin: /content/drive/MyDrive/Datasets/corpus_entrainement/

#@markdown ---

#@markdown #### Entrer le chemin vers un dossier où sauvegarder le modèle entraîné  :
chemin_destination_modele = "" #@param {type:"string"}
#@markdown Exemple de chemin: /content/drive/MyDrive/Modeles/modele/

#@markdown Le dossier est créé si non-existant

#@markdown ---

#@markdown #### Spécifier les paramètres d'entraînement :

#@markdown Nombre d'itérations :
MAX_ITER =  300#@param {type:"integer"}
#@markdown Fréquence de sauvegarde du modèle en nombre d'itérations :
CHECKPOINT_PERIOD =  50#@param {type:"integer"}
#@markdown Taux d'apprentissage (décimal compris entre 0 et 1) :
LEARNING_RATE = 0.00025 #@param {type:"number"}
#@markdown Apprentissage à partir d'un modèle préalablement entraîné :

#@markdown Entrer le chemin vers le modèle .pth ou laisser vide pour partir du modèle pré-entraîné PRIMA
MODELE = "" #@param {type:"string"}
#@markdown Exemple de chemin: /content/drive/MyDrive/Modeles/modele/model_final.pth


#@markdown ---


if not os.path.exists(chemin_destination_modele) :
  os.makedirs(chemin_destination_modele)

# Transformation du fichier configuration pour l'entrainement du modele


with open("/content/layout-model-training/config_LayoutParser_PrimaDataset_original.yaml", "r") as c :
  conf = c.read()
conf = conf.replace("MAX_ITER: 60000","MAX_ITER: {}".format(MAX_ITER))
conf = conf.replace("CHECKPOINT_PERIOD: 5000","CHECKPOINT_PERIOD: {}".format(CHECKPOINT_PERIOD)) 
conf = conf.replace("BASE_LR: 0.00025","BASE_LR: {}".format(LEARNING_RATE))
if MODELE :
  conf = conf.replace("WEIGHTS: https://www.dropbox.com/s/h7th27jfv19rxiy/model_final.pth?dl=1","WEIGHTS: {}".format(MODELE))
with open("/content/layout-model-training/config_LayoutParser_PrimaDataset.yaml", "w") as c :
  c.write(conf)




# Lancement de l'entraînement

ds_name = chemin_destination_modele.split("/")[-1]
annot_train = os.path.join(chemin_corpus,"train","*.json")
img_train = os.path.join(chemin_corpus,"train")
annot_val = os.path.join(chemin_corpus,"val","*.json")
img_val = os.path.join(chemin_corpus,"val")


!python /content/layout-model-training/tools/train_net.py --dataset_name $ds_name \
    --json_annotation_train $annot_train \
    --image_path_train      $img_train \
    --json_annotation_val   $annot_val \
    --image_path_val        $img_val \
    --config-file           /content/layout-model-training/config_LayoutParser_PrimaDataset.yaml \
    OUTPUT_DIR  $chemin_destination_modele \
    SOLVER.IMS_PER_BATCH 2


# Essai d'un modèle sur un petit corpus

In [5]:
#@markdown ## Essai du modèle 
#@markdown Renseigner les paramètres avant de lancer la cellule

#@markdown ---

#@markdown #### Entrer le chemin du modèle .pth à charger :
modele = "" #@param {type:"string"}
#@markdown Exemple de chemin: /content/drive/MyDrive/Modeles/modele/modele_final.pth

#@markdown ---

#@markdown #### Entrer le chemin du fichier de configuration .yaml de ce modèle  :
config = "" #@param {type:"string"}
#@markdown Exemple de chemin: /content/drive/MyDrive/Modeles/modele/config.yaml

#@markdown ---

#@markdown #### Entrer l'association id:nom pour chaque classe à détecter, séparées par un esapce (ou laisser vide et ne pas associer):

labels = "" #@param {type:"string"}

#@markdown Exemple : 0:illustration 1:legende 2:titre

try :
  if labels : 
    dictlabels = {}
    for asso in labels.split(" ") :
      dictlabels[asso.split(":")[0]] = asso.split(":")[1]

    model = lp.Detectron2LayoutModel(config_path= config,
                                  model_path=modele,
                                  label_map=dictlabels)
    print("Modèle importé")

  else : 
    model = lp.Detectron2LayoutModel(config_path= config,
                                      model_path=modele)
                                  #label_map=labels)
    print("Modèle importé")
except : print("Le modèle n'a pas été importé")

Modèle importé


In [ ]:
#@markdown ## Visualisation de l'inférence sur une page de document


#@markdown #### Entrer le chemin vers la page d'un document au format jpg :
page = "" #@param {type:"string"}
#@markdown Exemple de chemin: /content/drive/MyDrive/Datasets/Journal/page02.jpg

#@markdown ---

#@markdown #### Filtrer les résultats :

seuil_score = 0.3 #@param {type:"number"}

#@markdown 0 ou vide pour ne pas filtrer

image = cv2.imread(page)
image = image[..., ::-1]

layout = model.detect(image)
if seuil_score :
  layout = [x for x in layout if x.score > seuil_score]
  
for x in layout :
  print(x.type,x.score)

lp.draw_box(image, layout, box_width=3, show_element_id=True)
